# Sugarcane Harvest Front Scheduling Problem

This code is based of the work in [this](https://www.sciencedirect.com/science/article/abs/pii/S0925527319300957?via%3Dihub) paper. The authors are Rogerio de Ávila Ribeiro Junqueira and Reinaldo Morabito. 

## Abstract

Sugarcane harvest front scheduling is a complex process that affects the main activities of sugarmill supply logistics and it should simultaneously consider the available harvest and transport capacities, which are sometimes neglected. This paper models this scheduling problem, exploring an analogy to the well-known general lotsizing and scheduling problem with parallel production lines, and presents a heuristic method to solve it, involving an aggregation process, a constructive MIP heuristic based on relax-and-fix and an improving MIP heuristic based on fix-and-optimize. This optimization approach is applied to solve a large-sized real problem instance of a Brazilian mill whole season. To validate the approach, a validation process of the literature is applied by consulting a selected group of Brazilian harvest planning specialists. The results highlight the potential of the approach and how it can change the current harvest planning practices, enabling plans that consider the balance of harvest and transport capacities without using sectorization

In [ ]:
import pandas as pd
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gurobipy as gp
from gurobipy import GRB

***

## Model Parameters

$$\begin{align}

%% 1 
& \text{Harvesting blocks} & : \qquad & j =  1,...B \\
%% 2
& \text{Estimated production of the blocks (in tons)} &: \qquad & p_j  \\
%% 3
& \text{Harvesting Fronts} & : \qquad & l =  1,...,F \\
%% 4
& \text{Macro Periods} & : \qquad & t =  1,..,T \\
%% 5
& S_t \text{ (contains the micro-periods of t)} & : \qquad & s = 1,..,\sum_{t=1}^T|S_t| = N \\
%% 6
& \text{the first  micro-period of each t} & : \qquad& SO_t \\
%% 7
& \text{Blocks that can be harvested by front } l & : \qquad & Bl_{jl} \\
%% 8
& \text{Time Windows to harvest the blocks} & : \qquad & Bs_{jt} \\
%% 9
& \text{Harvesting type (manual or mechanized)} & : \qquad & m \in M \\
%% 10
& \text{Set of fronts that belong to } M & : \qquad & F_m \\
%% 11
& \text{Fraction of the production of j using m} & : \qquad & f_{jm}\\\
%% 12
& \text{Sugarcane meal demand range for each harvesting} & : \qquad & [mind_{mt}, maxd_{mt}] \\
%% 13
& \text{Minimum amount of area to be released} & : \qquad & vin_t \in V_j \\
%% 14
& \text{Fraction of a block that can be irrigated by vinasse} & : \qquad & fi_j \\
%% 15
& \text{Set of j's that have a fraction} fl_j \text{that can be irrigated by vinasse}& : \qquad & V_J \\
%% 16
& \text{Productivity of j (in tons per hec)}& : \qquad & TCH_j \\
%% 17
& \text{Machines in front l}& : \qquad & Nm_l \\
%% 18
& \text{harvesting capacity of block j (tons/hour)}& : \qquad & col_{mj} \\
%% 19
& \text{Capacity (continous hours/day) of the machines }m & : \qquad & Ht_m \\
%% 20
& \text{Fleet of homogeneous trucks} & : \qquad &  N_t \\
%% 21
& \text{Capacity (continuos hours per day) for the trucks} & : \qquad & Htt \\
%% 22
& \text{Working availability for the fleet of vehicles for each t} & : \qquad & K_t \\
%% 23
& \text{Transport capacity (tons/hour)of the block j that has the harvesting type }m & : \qquad & transp_{mj} \\
%% 24
& \text{Displacement time from block } i \text{ to } j & :\qquad & st_{ij} \\
%% 25
& \text{Distance between blocks} & : \qquad & dist_{ij} \\
%% 26
& \text{Number of vehicles with a flatbed trailer} & : \qquad & Np \\
%% 27
& \text{Minimum amount to be harvested from each block from each front} & : \qquad & bm_{lj} \\
%% 28
& \text{Opportunity Cost of  a non-milled ton of sugarcane when } mind_{mt} \text{ is not met} & : \qquad & mo \\
%% 29
& \text{Opportunity cost of a ton of sugarcane left unharvested} & : \qquad & bs \\
%% 30
& \text{Unit cost of the distance travelled by the front moving between blocks} & : \qquad & md \\

\end{align}$$

## Decision variables 

$$\begin{align}

& \text{(Positive Real) Production of block } j \text{ during micro-period } s \text{ by front } l & : \qquad & x_{ljs} \\
& \text{(Binary) If a front is in a block during a microperiod} & : \qquad & y_{ljs} \\
& \text{(Binary) If a front is moving between blocks } i \text{ and } j \text{during a micro-period} & : \qquad & z_{lijs} \\
& \text{(Positive Real) Amount of tons of milling lost from } m \text{ harvest during period } t & : \qquad & wm_{mt} \\
& \text{(Positive Real) amount of tons of raw material left in block } j & : \qquad & wb_j

\end{align}$$


#### Footnotes

Macro-periods: the divisions of the planning horizon. Each macroperiod can be sub-divided into non-overlaping micro-periods of different sizes. Each harvesting front has a specific definition of sub-periods


## Data For the model

In [ ]:
# size_B = 2
# size_F = 2
# size_T = 2
# N = 2
# S_t = {t: [s + N*(t-1)   for s in range(1,N+1)] for t in range(1,size_T+1)}
# SO_t = {t: [S_t[t][0]] for t in range(1,size_T+1)}
# V_j = {1}
# F_m = {'mechanized': [1],'manual':[2]}

# V_j = sorted(V_j)  # Convert to a sorted list
# F_m = {key: sorted(value) for key, value in F_m.items()}  # Convert values to sorted lists
# Blj = {1: [1,2],2:[1,2]}
# Bsj = {1: [1,2],2:[1,2]}

size_B = 6 # number of blocks
size_F = 3 # number of fronts
size_T = 4 # number o macro-periods
N = 3 # number of subperiods per period
S_t = {t: [s + N*(t-1)   for s in range(1,N+1)] for t in range(1,size_T+1)}
SO_t = {t: [S_t[t][0]] for t in range(1,size_T+1)}
V_j = {1,2,3,4,5,6}
F_m = {'mechanized': [1,2],'manual':[3]}

V_j = sorted(V_j)  # Convert to a sorted list
F_m = {key: sorted(value) for key, value in F_m.items()}  # Convert values to sorted lists
Blj = {1: [1,2,3,4,5,6],2:[1,2,3,4,5,6],3:[1,2,3,4,5,6]} # which front can harvest each block
Bsj = {1: [1,2,3,4],2:[1,2,3,4], 3:[2,3,4,5,6], 4:[3,4,5,6]} # blocks that can be harvested on each period

# print(S_t)
# print(SO_t)
# print(V_j)
# print(F_m)
# print(Blj)
# print(Bsj)


## Declaring parameters in pyomo

In [ ]:
model = pyo.ConcreteModel()
# Sets
model.B = pyo.RangeSet(1, size_B)  # Harvesting blocks
model.F = pyo.RangeSet(1, size_F)  # Harvesting fronts
model.T = pyo.RangeSet(1, size_T)  # Macro periods
model.S = pyo.RangeSet(1, N*size_T)  # Micro-periods
model.M = pyo.Set(initialize=['mechanized','manual']) # Harvesting type (manual or mechanized)
model.Vj = pyo.Set(initialize=V_j) # Set of j's that have a fraction fl_j that can be irrigated by vinasse
model.Fm = pyo.Set(model.M, initialize=F_m) # fronts belonging to M
model.St = pyo.Set(model.T, initialize=S_t) # micro periods belong to t
model.SOt = pyo.Set(model.T, initialize=SO_t) # first micro period of each t
model.Blj = pyo.Set(model.F,initialize=Blj)
model.Bsj = pyo.Set(model.T,initialize=Bsj)


B,F,T,S,M,Vj,Fm,St,SOt = model.B, model.F, model.T, model.S, model.M, model.Vj, model.Fm, model.St, model.SOt


In [ ]:
# p = {1:60,2:80}

# f = {1: {'mechanized': .5,'manual':.5},2: {'mechanized':.5,'manual': .5}}

# mind = {'mechanized': {1:5 ,2:20},'manual':{1:15 ,2:10}}
# maxd = {'mechanized': {1:40,2:40},'manual':{1:30 ,2:30}}

# vin = {1: 30,2:20}
# fi = {1: 1,2:1}
# TCH = {1: 1,2:1}

# Nm = {1: 10,2:10}

# col = {'mechanized':{1:(10),2:(10)},'manual':{1:(5),2:(5)}}

# Ht = {'mechanized': 8, 'manual':8}
# Nt = {1: 16,2:16}
# Htt = 10

# K = {1: 2*24*2*1.5, 2: 2*24*2*1.5}
# transp =  {'mechanized': {1:10,2:10},'manual':{1:10,2:10}}

# st = {1: {1: 0,2: 3}, 2:{1:3,2:0}}
# dist = {1: {1: 0,2:100},2:{1:100,2:0}}

# Np = 7

# bm = {1: {1: 0,2: 0}, 2:{1: 0,2: 0}}

# mo = 100000
# bs = 10000
# md = 100

p = {1:1000,2:1500, 3:5000, 4:4000, 5:2000, 6:400}

f = {1: {'mechanized': 1,'manual':.0},2: {'mechanized':1,'manual': 0},
     3: {'mechanized': 1,'manual':.0},4: {'mechanized':1,'manual': 0},
     5: {'mechanized': 1,'manual':.0},6: {'mechanized':0,'manual': 1}}

mind = {'mechanized': {1:2000 ,2:2000,3:2000,4:2000},'manual':{1:0 ,2:0,3:0,4:400}}
maxd = {'mechanized': {1:5000 ,2:3000,3:4000,4:3000}, 'manual': {1:400  ,2:400,3:400,4:800}}

vin = {1: 15/3,2:15/3,3:30/3,4:20/3}
fi = {1: 0.9,2:1,3:0.95,4:0.9,5:0.8,6:1}
TCH = {1: 100,2:80,3:100,4:70,5:60,6:60} # 80 ton/hec na media

Nm = {1: 30,2:20,3:10}

col = {'mechanized':{1:(59),2:(79),3:(100),4:(80),5:(90),6:(1)},'manual':{1:(1),2:(1),3:(1),4:(1),5:(1),6:15}}

Ht = {'mechanized': 10, 'manual':10}
Nt = {1: 18,2:10,3:10,4:12}
Htt = 10

K = {1: 36*8*3*8, 2: 36*8*3*8,3: 36*8*3*8, 4:36*8*3*8}
transp =  {'mechanized': {1:1.5*80,2:1.5*60,3:1.5*90,4:1.5*85,5:1.5*100,6:1.5*1},'manual':{1:1,2:1,3:1,4:1,5:1,6:40}}


dist = {
    1: {1: 0,  2: 5,  3: 10, 4: 15, 5: 20, 6: 25},
    2: {1: 5,  2: 0,  3: 6,  4: 11, 5: 16, 6: 21},
    3: {1: 10, 2: 6,  3: 0,  4: 7,  5: 12, 6: 17},
    4: {1: 15, 2: 11, 3: 7,  4: 0,  5: 8,  6: 13},
    5: {1: 20, 2: 16, 3: 12, 4: 8,  5: 0,  6: 9},
    6: {1: 25, 2: 21, 3: 17, 4: 13, 5: 9,  6: 0}
}

st = {
    1: {1: 0,         2: 0.125, 3: 0.25,  4: 0.375, 5: 0.5,   6: 0.625},
    2: {1: 0.125,     2: 0,     3: 0.15,  4: 0.275, 5: 0.4,   6: 0.525},
    3: {1: 0.25,      2: 0.15,  3: 0,     4: 0.175, 5: 0.3,   6: 0.425},
    4: {1: 0.375,     2: 0.275, 3: 0.175, 4: 0,     5: 0.2,   6: 0.325},
    5: {1: 0.5,       2: 0.4,   3: 0.3,   4: 0.2,   5: 0  ,   6: 0.225},
    6: {1: 0.625,     2: 0.525, 3: 0.425, 4: 0.325, 5: 0.225, 6: 0}
} # Velocidade media de 40km/h para as distâncias. St está em horas

Np = 30
bm = {
    1: {1: 20*25, 2: 15*25, 3: 10*25, 4: 25*25, 5: 18*25, 6: 1000},  # Frente 1 (mecanizada)
    2: {1: 20*25, 2: 15*25, 3: 10*25, 4: 25*25, 5: 18*25, 6: 1000},  # Frente 2 (mecanizada)
    3: {1:100, 2:100, 3:100, 4:100, 5:100, 6: 10}         # Frente 3 (manual)
}

mo = 100_000
bs = 10_000
md = 5_000


In [ ]:
# p = {1:60,2:80}

# f = {1: {'mechanized': .5,'manual':.5},2: {'mechanized':.5,'manual': .5}}

# mind = {'mechanized': {1:5 ,2:20},'manual':{1:15 ,2:10}}
# maxd = {'mechanized': {1:40,2:40},'manual':{1:30 ,2:30}}

# vin = {1: 30,2:20}
# fi = {1: 1,2:1}
# TCH = {1: 1,2:1}

# Nm = {1: 10,2:10}

# col = {'mechanized':{1:(10),2:(10)},'manual':{1:(5),2:(5)}}

# Ht = {'mechanized': 8, 'manual':8}
# Nt = {1: 16,2:16}
# Htt = 10

# K = {1: 2*24*2*1.5, 2: 2*24*2*1.5}
# transp =  {'mechanized': {1:10,2:10},'manual':{1:10,2:10}}

# st = {1: {1: 0,2: 3}, 2:{1:3,2:0}}
# dist = {1: {1: 0,2:100},2:{1:100,2:0}}

# Np = 7

# bm = {1: {1: 0,2: 0}, 2:{1: 0,2: 0}}

# mo = 100000
# bs = 10000
# md = 100

p = {1:1000,2:1500, 3:5000, 4:4000, 5:2000, 6:400}

f = {1: {'mechanized': 1,'manual':.0},2: {'mechanized':1,'manual': 0},
     3: {'mechanized': 1,'manual':.0},4: {'mechanized':1,'manual': 0},
     5: {'mechanized': 1,'manual':.0},6: {'mechanized':0,'manual': 1}}

mind = {'mechanized': {1:2000 ,2:2000,3:2000,4:2000},'manual':{1:0 ,2:0,3:0,4:400}}
maxd = {'mechanized': {1:5000 ,2:3000,3:4000,4:3000}, 'manual': {1:400  ,2:400,3:400,4:800}}

vin = {1: 15/3,2:15/3,3:30/3,4:20/3}
fi = {1: 0.9,2:1,3:0.95,4:0.9,5:0.8,6:1}
TCH = {1: 100,2:80,3:100,4:70,5:60,6:60} # 80 ton/hec na media

Nm = {1: 30,2:20,3:10}

col = {'mechanized':{1:(59),2:(79),3:(100),4:(80),5:(90),6:(1)},'manual':{1:(1),2:(1),3:(1),4:(1),5:(1),6:15}}

Ht = {'mechanized': 10, 'manual':10}
Nt = {1: 18,2:10,3:10,4:12}
Htt = 10

K = {1: 36*8*3*8, 2: 36*8*3*8,3: 36*8*3*8, 4:36*8*3*8}
transp =  {'mechanized': {1:1.5*80,2:1.5*60,3:1.5*90,4:1.5*85,5:1.5*100,6:1.5*1},'manual':{1:1,2:1,3:1,4:1,5:1,6:40}}


dist = {
    1: {1: 0,  2: 5,  3: 10, 4: 15, 5: 20, 6: 25},
    2: {1: 5,  2: 0,  3: 6,  4: 11, 5: 16, 6: 21},
    3: {1: 10, 2: 6,  3: 0,  4: 7,  5: 12, 6: 17},
    4: {1: 15, 2: 11, 3: 7,  4: 0,  5: 8,  6: 13},
    5: {1: 20, 2: 16, 3: 12, 4: 8,  5: 0,  6: 9},
    6: {1: 25, 2: 21, 3: 17, 4: 13, 5: 9,  6: 0}
}

st = {
    1: {1: 0,         2: 0.125, 3: 0.25,  4: 0.375, 5: 0.5,   6: 0.625},
    2: {1: 0.125,     2: 0,     3: 0.15,  4: 0.275, 5: 0.4,   6: 0.525},
    3: {1: 0.25,      2: 0.15,  3: 0,     4: 0.175, 5: 0.3,   6: 0.425},
    4: {1: 0.375,     2: 0.275, 3: 0.175, 4: 0,     5: 0.2,   6: 0.325},
    5: {1: 0.5,       2: 0.4,   3: 0.3,   4: 0.2,   5: 0  ,   6: 0.225},
    6: {1: 0.625,     2: 0.525, 3: 0.425, 4: 0.325, 5: 0.225, 6: 0}
} # Velocidade media de 40km/h para as distâncias. St está em horas

Np = 30
bm = {
    1: {1: 20*25, 2: 15*25, 3: 10*25, 4: 25*25, 5: 18*25, 6: 1000},  # Frente 1 (mecanizada)
    2: {1: 20*25, 2: 15*25, 3: 10*25, 4: 25*25, 5: 18*25, 6: 1000},  # Frente 2 (mecanizada)
    3: {1:100, 2:100, 3:100, 4:100, 5:100, 6: 10}         # Frente 3 (manual)
}

mo = 100_000
bs = 10_000
md = 5_000


In [ ]:
# Parameters
# Parameters (commented again so i dont get lost)

model.p = pyo.Param(model.B,initialize=p, within=NonNegativeReals)  # Estimated production of the blocks (in tons)


def init_f(model,j,m):
    return f[j][m]
model.f = pyo.Param(model.B, model.M,initialize=init_f, within=NonNegativeReals)  # Fraction of the production of j using m

def init_mind(model,m,j):
    return mind[m][j]
model.mind = pyo.Param(model.M, model.T,initialize=init_mind, within=NonNegativeReals)  # Sugarcane meal demand range (min)

def init_maxd(model,m,j):
    return maxd[m][j]
model.maxd = pyo.Param(model.M, model.T,initialize=init_maxd, within=NonNegativeReals)  # Sugarcane meal demand range (max)


model.vin = pyo.Param(model.T, initialize=vin, within=NonNegativeReals)  # Minimum amount of area to be released
model.fi = pyo.Param(model.B,initialize=fi ,within=NonNegativeReals)  # Fraction of a block that can be irrigated by vinasse
model.TCH = pyo.Param(model.B,initialize=TCH, within=NonNegativeReals)  # Productivity of j (in tons per hec)

def init_Nm(model,l):
    return Nm[l]
model.Nm = pyo.Param(model.F,initialize=init_Nm ,within=NonNegativeIntegers)  # Machines in front l

def init_col(model,m,j):
    return col[m][j]
model.col = pyo.Param(model.M,model.B,initialize=init_col, within=NonNegativeReals)  # Harvesting capacity of block j (tons/hour)
model.Ht = pyo.Param(model.M,initialize=Ht ,within=NonNegativeReals)  # Capacity (continuous hours/day) of the machines m
model.Nt = pyo.Param(model.T,initialize=Nt,within=NonNegativeIntegers)  # Fleet of homogeneous trucks
model.K = pyo.Param(model.T,initialize=K ,within=NonNegativeReals)  # Working availability for the fleet of vehicles for each t

def init_transp(model,m,j):
    return transp[m][j]
model.transp = pyo.Param(model.M,model.B,initialize=init_transp, within=NonNegativeReals)  # Transport capacity (tons/hour) of the block j

def init_st(model,i,j):
    if i != j:
        return st[i][j]
    else:
        return 0

model.st = pyo.Param(model.B, model.B,initialize=init_st, within=NonNegativeReals)  # Displacement time from block i to j

def init_dist(model,i,j):
    if i !=j:
        return dist[i][j]
    else:
        return 0
model.dist = pyo.Param(model.B, model.B,initialize=init_dist, within=NonNegativeReals)  # Distance between blocks

def init_bm(model,l,j):
    return bm[l][j]
model.bm = pyo.Param(model.F, model.B,initialize=init_bm, within=NonNegativeReals)  # Minimum amount to be harvested from each block from each front

model.Htt = pyo.Param(initialize=Htt,within=NonNegativeReals )  # Capacity (continuous hours per day) for the trucks
model.Np = pyo.Param(initialize=Np,within=NonNegativeIntegers)  # Number of vehicles with a flatbed trailer
model.mo = pyo.Param(initialize=mo, within=NonNegativeReals)  # Opportunity cost of a non-milled ton of sugarcane when mind_{mt} is not met
model.bs = pyo.Param(initialize=bs, within=NonNegativeReals)  # Opportunity cost of a ton of sugarcane left unharvested
model.md = pyo.Param(initialize=md, within=NonNegativeReals)  # Unit cost of the distance travelled by the front moving between blocks


p,Blj,Bsj,f,mind,maxd,vin,fi,TCH,Nm,col,Ht,Nt,K,transp,st,dist,bm,Htt,Np,mo,bs,md = model.p, model.Blj, model.Bsj, model.f, model.mind, model.maxd, model.vin, model.fi, model.TCH, model.Nm, model.col, model.Ht, model.Nt, model.K, model.transp, model.st, model.dist, model.bm, model.Htt, model.Np, model.mo, model.bs, model.md


In [ ]:
# Decision Variables
def x_bounds(model,l, j,s):
    return (0, model.p[j])

model.x = pyo.Var(model.F, model.B, model.S, within=NonNegativeReals,bounds=x_bounds)
model.y = pyo.Var(model.F, model.B, model.S, within=Binary)
model.z = pyo.Var(model.F, model.B, model.B, model.S, within=Binary)

def wm_bounds(model, m, j):
    return (0, model.mind[m, j])
model.wm = Var(model.M, model.T, bounds=wm_bounds)

def wb_bounds(model, j):
    return (0, model.p[j])
model.wb = Var(model.B, bounds=wb_bounds)
x,y,z,wm,wb = model.x,model.y,model.z,model.wm,model.wb


***
## Model


### Objective Function

Objective function to minimize the total cost, which includes:
- Opportunity cost of milling lost from each harvest type during each period.I.e: Cost of milling failure (junqueira, morabito)
- Opportunity cost of tons of sugarcane left unharvested. (junqueira, morabito)
- Unit cost of the distance traveled by the front moving between blocks. I.e: displacement cost (junqueira, morabito)

$$
\begin{align}

% 1
& Min \qquad mo \sum_{t=1}^T wm_{mt} + bs \sum_{j=1}^B wb_j + md \sum_{l-1}^F \sum_{j-1}^B \sum_{j=1}^B (dist_{ij} \cdot z_{lij}) \\

\end{align}
$$

### Subject to:
$$
\begin{align}
%2
&\sum_{l \in F_m} \sum_{j=1}^{B} \sum_{s \in S_{t}} x_{ljs} + wm_{mt} \geq mind_{mt} &  \quad (t = 1, \ldots, T), \quad (m = man, mec)  \tag{2}\\
%3
&\sum_{l \in F_m} \sum_{j=1}^{B} \sum_{s \in S_{t}} x_{ljs} \leq maxd_{mt} & \quad (t = 1, \ldots, T), \quad (m = man, mec) \tag{3}\\
%4
&\sum_{l \in F_m} \sum_{s=1}^{N} x_{ljs} + wb_{j} = p_j f_{jm} & \quad (j = 1, \ldots, B), \quad (m = man, mec) \tag{4}\\
%5
&\sum_{m=man}^{mec} \sum_{l \in F_m} \sum_{j \in V_j}^{B} \sum_{s \in S_{t}} fi_j \frac{x_{ljs}}{TCH_{j}} \geq vin_t & \quad (t = 1, \ldots, T) \tag{5}\\
%6
&\sum_{j=1}^{B} \sum_{s \in S_t} \frac{24}{col_{mj} Nm_l Ht_{m}} x_{ljs} + \sum_{i=1}^{B} \sum_{j=1}^B \sum_{s \in S_t} \frac{Nm_l}{Np} st_{ij} z_{lijs} \leq K_{t} & \quad (m = man, mec), \quad \forall l \in F_{m}, (t = 1, \ldots, T) \tag{6}\\
%7
&\sum_{m=man}^{mec} \sum_{l \in F_m} \sum_{j=1}^{B} \sum_{s \in S_{t}} \frac{24}{transp_{mj}NtHtt} x_{ljs} \leq K_{t} & \quad (t = 1, \ldots, T) \tag{7}\\
%8
&x_{ljs} \leq \min \left( \frac{col_{mj}Nm_L Ht_{m}}{24}, \frac{transp_{mj}NtHtt}{24} \right) K_{t} y_{ljs} & \quad (m = man, mec),  \quad \forall l \in F_{m}, (j = 1, \ldots, B), (t = 1, \ldots, T), \quad \forall s \in S_{t} \tag{8}\\
%9
&x_{ljs} \geq bm_{lj}(y_{ljs} - y_{ljs-1}) & \quad (m = man, mec), \quad \forall l \in F_{m}, (j = 1, \ldots, B), \quad (s=1,\ldots,N) \tag{9}\\
%10
& \sum_{j \in (Bs_{jt} \cap Bl_jl )}y_{ljs} = 1 & \quad (l = 1, \ldots, F), (t = 1, \ldots, T), \quad \forall s \in S_{t}\tag{10}\\
%11
&y_{ljs-1} = \sum_{j=1}^{B} z_{lijs} & \quad (l = 1, \ldots, F), (i = 1, \ldots, B), (s = 2, \ldots, N) \tag{11}\\
%12
&\sum_{i=1}^{B} z_{lijs} = y_{ljs} & \quad (l = 1, \ldots, F), (j = 1, \ldots, B), (s = 1, \ldots, N) \tag{12}\\
%13
&y_{ljs-1} \geq y_{ljs} & \quad (l = 1, \ldots, F) \quad (j = 1, \ldots, B), (t = 1, \ldots, T), \forall s \in (S_{t} \setminus S_{0}) \tag{13}\\
%14
&x_{ljs} \geq 0 & \quad (l = 1, \ldots, F), (j = 1, \ldots, B), (s = 1, \ldots, N) \tag{14}\\
%15
&y_{ljs} \in \{0,1\} & \quad (l = 1, \ldots, F), (j = 1, \ldots, B), (s = 1, \ldots, N) \tag{15}\\
%16
&z_{lijs} \geq 0 & \quad (l = 1, \ldots, F), (i = 1, \ldots, B), (j = 1, \ldots, B), (s = 1, \ldots, N) \tag{16}\\
%17
&wm_{mt} \geq 0 & \quad (t = 1, \ldots, T), (m = man, mec) \tag{17}\\
%18
&wb_{j} \geq 0 & \quad (j = 1, \ldots, B) \tag{18}

\end{align}
$$

### Definitions in Python

In [ ]:

#2
def meet_minimum_demand(model,list_of_constraints,t,m):
    """
    Constraint to ensure that the production and milling lost meets the minimum demand. (junqueira, morabito)

    Args:
        model (ConcreteModel): The Pyomo model instance containing all sets, parameters, and variables.
        t (int): The current macro-period.
        m (str): The type of harvesting (manual or mechanized).

    Returns:
        Constraint: "Ensures that the sum of production by all fronts and milling lost is
        greater than or equal to the minimum demand for each harvesting type during each period." (junqueira, morabito)
    """

    model.minimum_demand_list.add(expr=(sum(x[l, j, s] for s in St[t] for j in B for l in Fm[m]) + wm[m, t] >= mind[m, t]))

#3
def stop_at_maximum_demand(model,list_of_constraints,t,m):
    """
    Constraint to ensure that the production does not exceed the maximum demand. (junqueira, morabito)

    Args:
        model (ConcreteModel): The Pyomo model instance containing all sets, parameters, and variables.
        t (int): The current macro-period.
        m (str): The type of harvesting (manual or mechanized).

    Returns:
        Constraint: "Ensures that the sum of production by all fronts does not exceed the
        maximum demand for each harvesting type during each period." (junqueira, morabito)
    """

    model.maximum_demand_list.add(expr=(sum(x[l, j, s] for s in St[t] for j in B for l in Fm[m])  <= maxd[m, t]))

#4
def limit_harvest_and_transport(model,list_of_constraints,j,m):
    """
    Constraint to limit the harvest and transport to the availability of raw material in the block (junqueira, morabito)

    Args:
        model (ConcreteModel): The Pyomo model instance containing all sets, parameters, and variables.
        t (int): The current macro-period.
        m (str): The type of harvesting (manual or mechanized).

    Returns:
        Constraint: "limit the harvest and transport to the availability of raw material in the block and are, therefore, related to the field production, considering the kind of harvesting of the front and the block."(junqueira, morabito)
    """
    model.limit_harvest_and_transpot_list.add(expr=(sum(x[l,j,s] for s in S for l in Fm[m]) + wb[j] == p[j]*f[j,m]))

#5
def minimum_amount_of_vinasse(model,list_of_constraints,t):
    """
    Constraint to Determine the minimum amount of vinasse area to be released on the macro periods (junqueira, morabito)

    Args:
        model (ConcreteModel): The Pyomo model instance containing all sets, parameters, and variables.
        t (int): The current macro-period.

    returns:
        Constraint: "determine the minimum amount of vinasse area to be released at each macro-period t, taking into account the amount of irrigable area in the block" (Junqueira, Morabito)
    """

    model.minimum_amount_of_vinasse_list.add(expr=(sum((x[l,j,s]/TCH[j])*fi[j] for s in St[t] for l in F for j in Vj) >= vin[t]))

#6
def limit_harvesting_capacity(model,list_of_constraints,t,m,l):
    """
    Constraint to relate the time spent in producing harvest resources and the time spent displacing the equipament and the capacity K_t, the working availabily of the fleet for each macro-period

    Args:
        model (ConcreteModel): The Pyomo model instance containing all sets, parameters, and variables.
        t (int): The current macro-period.
        m (str): The type of harvesting (manual or mechanized).
        l (int): the harvesting front

    returns:
        Constraint: "relate the time spent producing harvest resources, as well as the time spent on transporting equipment from one area to another with the total time available in the period. The resource production time takes into account the block performance characteristics, as well as the agronomic suitability for the harvest during the period. The time for transporting the equipment, when moving from one area to another, takes into account the time spent on this move, as well as the number of vehicles with a flatbed trailer available for this operation." (Junqueira, Morabito)

    """

    model.limit_harvesting_capacity_list.add(expr=( sum(((24)/(col[m,j]*Nm[l]*Ht[m]))*x[l,j,s] for s in St[t] for j in B) + sum( (Nm[l]/Np)*st[i,j]*z[l,i,j,s] for s in St[t] for j in B for i in B) <= K[t]))

#7
def limit_transport_capacity(model,list_of_constraints,t):
    """
    Constraint to limit the transportation resources

    args:
        model (ConcreteModel): The Pyomo model instance containing all sets, parameters, and variables.
        t (int): The current macro-period.

    returns:
        constraint: "ddress the trans- portation resources, whose production potential is considered per block. In this case, the time spent on moving the front is not inserted because, when changing areas, these resources can either transport the production of the areas under operation, or they are idle consuming stock on the roads at the agroindustrial unit" (Junqueira, Morabito)
    """
    model.limit_transport_capacity_list.add(expr=(sum((24/(transp[m,j]*Nt[t]*Htt))*x[l,j,s] for s in St[t] for j in B for m in M for l in Fm[m]) <= K[t]))

#8
def limit_production_to_capacity(model,list_of_constraints,t,s,j,m,l):
    """
    Constraint to couple x and y

    args:
        model (ConcreteModel): The Pyomo model instance containing all sets, parameters, and variables.
        m (str): The type of harvesting (manual or mechanized).
        j (int): The harvesting block
        l (int): the harvesting front
        s (int): The current micro-period.

    Returns:
        Constraint:"ensure that when there is production in block j by the harvesting front l during micro-period s, the front is positioned at the same place and time. In the same way, for the opposite situation, when the front is not positioned, there is no production of the block. It is important to mention that the upper limit of x_ljs was considered as the minimum between the harvesting and transport capacity" (Junqueira, Morabito)

    """
    # print((transp[m,j]*Nt[t]*Htt)/24)
    # print(((col[m,j]*Nm[l]*Ht[m])/24))
    # print(min(((transp[m,j]*Nt[t]*Htt)/24),((col[m,j]*Nm[l]*Ht[m])/24)))
    model.limit_production_to_capacity_list.add(expr=(x[l,j,s]<= min((transp[m,j]*Nt[t]*Htt)/24,(col[m,j]*Nm[l]*Ht[m])/24)*K[t]*y[l,j,s]))

#9
def floor_of_production_capacity(model,list_of_constraints,l,j,s):
    """
    Constraint to define the minum lot size

    args:
        model (ConcreteModel): The Pyomo model instance containing all sets, parameters, and variables.
        m (str): The type of harvesting (manual or mechanized).
        j (int): The harvesting block
        l (int): the harvesting front
        s (int): The current micro-period.

    Returns:
        Constraint:Constraint to define the minum lot size (Junqueira, Morabito)
    """

    model.floor_of_production_capacity_list.add(expr=(x[l,j,s] >= bm[l,j]*(y[l,j,s] - y[l,j,s-1])))

#10
def harvest_all_blocks(model,list_of_constraints,t,s,l):
    """
    Constraint to make at least one block to be visited on micro period s and that the block is visited in the correct time window.

    args:
        model (ConcreteModel): The Pyomo model instance containing all sets, parameters, and variables.
        l (int): the harvesting front
        s (int): The current micro-period.

    returns:
        Constraint: "ensure that front l will be in only one block j during micro-period s. Using parameterBs jt , these constraints impose that the harvest occurs in blocks j allowed by the time windows,that is, during macro-periods t. In addition, using parameterBl jl , these equations define blocks j that a harvesting front l can harvest. This happens when it is necessary to sectorize one of the fronts. It is worth highlighting that=y 0ljs are previously fixed for micro-periods s that are out of the time window of block j (Bs )jt , as well as for those fronts l that cannot be harvested (Bl )jl" (Junqueira, morabito)
    """
    valid_blocks = set(value for value in Bsj[t] if value in Blj[l])
    model.harvest_all_blocks_list.add(expr=(sum(y[l,j,s] for j in valid_blocks) == 1))

#11
def consistent_movement_on_period_s_minus(model,list_of_constraints,s,l,i):
    """
    Constraint to define the movement of the front in micro-period s-1

    args:
        model (ConcreteModel): The Pyomo model instance containing all sets, parameters, and variables.
        l (int): the harvesting front
        s (int): The current micro-period
        j (int): The harvesting block

    returns:
        Constraint: togheter with constraint #12 "define the movement of the front by variablez lijs , consistent with the positions of the front during micro-period s (yljs ) and during the previous micro- period s-1 (ylis 1 )." (Junqueira, Morabito)

    returns:
    """
    model.consistent_movement_on_period_s_minus_list.add(expr=(sum(z[l,i,j,s] for j in B) == y[l,i,s-1]))

#12
def consistent_movement_on_period_s(model,list_of_constraints,s,l,j):
    """
    Constraint to define the movement of the front in micro-period s-1

    args:
        model (ConcreteModel): The Pyomo model instance containing all sets, parameters, and variables.
        l (int): the harvesting front
        s (int): The current micro-period
        j (int): The harvesting block

    returns:
        Constraint: togheter with constraint #12 "define the movement of the front by variablez lijs , consistent with the positions of the front during micro-period s (yljs ) and during the previous micro- period s-1 (ylis 1 )." (Junqueira, Morabito)

    returns:
    """
    model.consistent_movement_on_period_s_list.add(expr=(sum(z[l,i,j,s] for i in B) == y[l,j,s]))

#13
def idle_micro_period(model,list_of_constraints,s,j,l):
    """
    Constraint to define that the idle micro-periods only occur at the end of the macro-period

    args:
        model (ConcreteModel): The Pyomo model instance containing all sets, parameters, and variables.
        l (int): the harvesting front
        s (int): The current micro-period
        j (int): The harvesting block
    """

    model.idle_micro_period_list.add(expr=(y[l,j,s-1] >= y[l,j,s]))


### Script to Optimize the model

The code was written like this to make it more didact, however most of the loops could be joined

In [ ]:


model.objective = Objective(expr=(mo * sum(wm[m, t] for m in M for t in T) +bs*sum(wb[j] for j in B) +md*sum(dist[i, j] * z[l, i, j,s] for l in F for i in B for j in B for s in S)), sense=minimize)

# Fixando valores impossiveis de y
for j in B:
  for t in T:
    for l in F:
      if not j in set(value for value in Bsj[t] if value in Blj[l]):
        for s in St[t]:
            y[l,j,s].fix(0)

#2 & 3
model.minimum_demand_list = ConstraintList()
model.maximum_demand_list = ConstraintList()
for t in T:
    for m in M:
        meet_minimum_demand(model,model.minimum_demand_list,t,m)
        stop_at_maximum_demand(model,model.maximum_demand_list,t,m)

model.limit_harvest_and_transpot_list = ConstraintList()
# 4
for j in B:
    for m in M:
        limit_harvest_and_transport(model,model.limit_harvest_and_transpot_list,j,m)

model.minimum_amount_of_vinasse_list = ConstraintList()
#5
for t in T:
    minimum_amount_of_vinasse(model,model.minimum_amount_of_vinasse_list,t)

model.limit_harvesting_capacity_list = ConstraintList()
#6
for t in T:
    for m in M:
        for l in Fm[m]:
            limit_harvesting_capacity(model,model.limit_harvesting_capacity_list,t,m,l)

model.limit_transport_capacity_list = ConstraintList()
#7
for t in T:
    limit_transport_capacity(model,model.limit_transport_capacity_list,t)


model.limit_production_to_capacity_list = ConstraintList()
#8
for t in T:
    for s in St[t]:
        for j in B:
            for m in M:
                for l in Fm[m]:
                    limit_production_to_capacity(model,model.limit_production_to_capacity_list,t,s,j,m,l)

model.floor_of_production_capacity_list = ConstraintList()
#9
for s in S:
  if s>1:
    for l in F:
      for j in B:
        floor_of_production_capacity(model,model.floor_of_production_capacity_list,l,j,s)


model.harvest_all_blocks_list = ConstraintList()
#10
for t in T:
   for s in St[t]:
      for l in F:
        harvest_all_blocks(model,model.harvest_all_blocks_list,t,s,l)


model.consistent_movement_on_period_s_minus_list = ConstraintList()
#11
for s in S:
    if s > 1:
       for l in F:
           for i in B:
                consistent_movement_on_period_s_minus(model,model.consistent_movement_on_period_s_minus_list,s,l,i)

model.consistent_movement_on_period_s_list = ConstraintList()
#12
for s in S:
    for l in F:
        for j in B:
            consistent_movement_on_period_s(model,model.consistent_movement_on_period_s_list,s,l,j)

model.idle_micro_period_list = ConstraintList()
#13
for t in T:
    for s in St[t] - SOt[t]:
        for j in B:
          for l in F:
            idle_micro_period(model,model.idle_micro_period_list,s,j,l)

***
## Results

In [ ]:
# Solve the model again
opt = pyo.SolverFactory('gurobi')

# Executar a solução novamente se necessário
results = opt.solve(model, tee=True)

# Print the objective value if the problem was solved successfully
if results.solver.termination_condition == pyo.TerminationCondition.optimal:
  print("objective ",model.objective())


In [ ]:
fronts = []
blocks = []
micro_periods = []
productions = []
y_lista = []

for l in F:  # Assuming F is the set of fronts
    for j in B:  # Assuming B is the set of blocks
        for s in S:  # Assuming S is the set of micro-periods
            fronts.append(l)
            blocks.append(j)
            micro_periods.append(s)
            productions.append(model.x[l, j, s].value)  # Extracting the value of each x[l, j, s]
            y_lista.append(model.y[l, j, s].value)


df_X = pd.DataFrame({
    'Front': fronts,
    'Block': blocks,
    'Micro-Period': micro_periods,
    'Production': productions,
    'Y': y_lista
})


# Criar subplots: cada frente terá uma linha, cada linha terá 2 gráficos
fig, axs = plt.subplots(nrows=len(model.F), ncols=2, figsize=(9, 3 * 3), constrained_layout=True)

# Iterar por cada frente
for i, front in enumerate(model.F):
    front_data = df_X[df_X['Front'] == front]

    # Gráfico de barras para 'Production'
    sns.barplot(x='Micro-Period', y='Production', hue='Block', palette='Set2', data=front_data, ax=axs[i][0])
    axs[i][0].set_title(f'Production for Front {front}')
    axs[i][0].set_xlabel('Micro-Period')
    axs[i][0].set_ylabel('Production')
    axs[i][0].legend(title='Block', loc='upper right')

    # Gráfico de dispersão para 'Y'
    sns.barplot(x='Micro-Period', y='Y', hue='Block', palette='Set2', data=front_data, ax=axs[i][1])
    axs[i][1].set_title(f'Y Value for Front {front}')
    axs[i][1].set_xlabel('Micro-Period')
    axs[i][1].set_ylabel('Y')
    axs[i][1].legend(title='Block', loc='upper right')

plt.savefig('apresentação/imagens/resultado_inst_1.png')
plt.show()